In [97]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
.appName('retail order management systems')\
.getOrCreate()

In [98]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [99]:
raw_orders = [
    ("ORD001","C001","Ravi"," Delhi ","Laptop","Electronics","45000","2024-01-05","Completed"),
    ("ORD002","C002","Sneha","Mumbai"," Mobile ","Electronics","32000","05/01/2024","Completed"),
    ("ORD003","C003","Aman","Bangalore","Laptop","Electronics","55000","2024/01/06","Completed"),
    ("ORD004","C004","Pooja","Delhi","Tablet"," Electronics ","","2024-01-07","Cancelled"),
    ("ORD005","C005","Neha","Chennai","Laptop","Electronics","48000","invalid_date","Completed"),
    ("ORD006","C006","Rahul","Mumbai","Mobile","Electronics",None,"2024-01-08","Completed"),
    ("ORD007","C007","Kiran","Bangalore","Tablet","Electronics","30000","2024-01-08","Completed"),
    ("ORD008","C008","Amit","Delhi","Laptop","electronics","45000","2024-01-09","Completed"),
    ("ORD009","C009","Priya"," Pune","Mobile","Electronics","28000","09-01-2024","Completed"),
    ("ORD010","C010","Suresh","Mumbai","Laptop","Electronics","55000","2024-01-10","Completed"),
    ("ORD010","C010","Suresh","Mumbai","Laptop","Electronics","55000","2024-01-10","Completed"),  # duplicate
    ("ORD011","C011","Meena","Chennai","Tablet","Electronics","31000","2024-01-11","Completed"),
    ("ORD012","C012","Arjun","Delhi","Mobile","Electronics","27000","2024/01/11","Completed"),
    ("ORD013","C013","Nikhil","Bangalore","Laptop","Electronics","60000","2024-01-12","Completed"),
    ("ORD014","C014","Rohit","Mumbai","Mobile","Electronics","invalid_price","2024-01-12","Completed"),
    ("ORD015","C015","Anita","Delhi","Tablet","Electronics","29000","2024-01-13","Completed"),
    ("ORD016","C016","Vikas","Chennai","Laptop","Electronics","52000","2024-01-13","Completed"),
    ("ORD017","C017","Sunita","Mumbai","Mobile","Electronics","33000","2024-01-14","Completed"),
    ("ORD018","C018","Deepak","Bangalore","Laptop","Electronics","58000","2024-01-14","Completed"),
    ("ORD019","C019","Pallavi","Delhi","Mobile","Electronics","26000","2024-01-15","Completed"),
    ("ORD020","C020","Manish","Mumbai","Tablet","Electronics","34000","2024-01-15","Completed")
]
columns = ["order_id", "customer_id", "name", "city", "product", "category", "price", "order_date", "order_status"]
raw_df = spark.createDataFrame(data=raw_orders, schema=columns)
raw_df.show()

+--------+-----------+-------+---------+--------+-------------+-------------+------------+------------+
|order_id|customer_id|   name|     city| product|     category|        price|  order_date|order_status|
+--------+-----------+-------+---------+--------+-------------+-------------+------------+------------+
|  ORD001|       C001|   Ravi|   Delhi |  Laptop|  Electronics|        45000|  2024-01-05|   Completed|
|  ORD002|       C002|  Sneha|   Mumbai| Mobile |  Electronics|        32000|  05/01/2024|   Completed|
|  ORD003|       C003|   Aman|Bangalore|  Laptop|  Electronics|        55000|  2024/01/06|   Completed|
|  ORD004|       C004|  Pooja|    Delhi|  Tablet| Electronics |             |  2024-01-07|   Cancelled|
|  ORD005|       C005|   Neha|  Chennai|  Laptop|  Electronics|        48000|invalid_date|   Completed|
|  ORD006|       C006|  Rahul|   Mumbai|  Mobile|  Electronics|         NULL|  2024-01-08|   Completed|
|  ORD007|       C007|  Kiran|Bangalore|  Tablet|  Electronics| 

#CLEANING & TRANSFORMATION TASKS (FOR STUDENTS)

1. Rename all columns to snake_case

In [100]:
import re

def to_snake_case(name):
    name = re.sub(r'\s+', '_', name)
    name = re.sub(r'(?<!^)(?=[A-Z])', '_', name).lower()
    return name

old_columns = raw_df.columns

new_columns = [to_snake_case(col_name) for col_name in old_columns]

df = raw_df
for old_name, new_name in zip(old_columns, new_columns):
    df = df.withColumnRenamed(old_name, new_name)

df.printSchema()
df.show(5)

root
 |-- order_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- city: string (nullable = true)
 |-- product: string (nullable = true)
 |-- category: string (nullable = true)
 |-- price: string (nullable = true)
 |-- order_date: string (nullable = true)
 |-- order_status: string (nullable = true)

+--------+-----------+-----+---------+--------+-------------+-----+------------+------------+
|order_id|customer_id| name|     city| product|     category|price|  order_date|order_status|
+--------+-----------+-----+---------+--------+-------------+-----+------------+------------+
|  ORD001|       C001| Ravi|   Delhi |  Laptop|  Electronics|45000|  2024-01-05|   Completed|
|  ORD002|       C002|Sneha|   Mumbai| Mobile |  Electronics|32000|  05/01/2024|   Completed|
|  ORD003|       C003| Aman|Bangalore|  Laptop|  Electronics|55000|  2024/01/06|   Completed|
|  ORD004|       C004|Pooja|    Delhi|  Tablet| Electronics |     |  2024

2. Add a column price_with_tax (18%)

In [101]:
from pyspark.sql.functions import when, col
from pyspark.sql.types import DoubleType

df = df.withColumn("price",
                 when(col("price").rlike("^[0-9]+$"),col("price").cast(DoubleType())).otherwise(None))

if "price_with_tax" in df.columns:
    df = df.drop("price_with_tax")

df = df.withColumn("price_with_tax", col("price") * 1.18)

df.show(5)

+--------+-----------+-----+---------+--------+-------------+-------+------------+------------+--------------+
|order_id|customer_id| name|     city| product|     category|  price|  order_date|order_status|price_with_tax|
+--------+-----------+-----+---------+--------+-------------+-------+------------+------------+--------------+
|  ORD001|       C001| Ravi|   Delhi |  Laptop|  Electronics|45000.0|  2024-01-05|   Completed|       53100.0|
|  ORD002|       C002|Sneha|   Mumbai| Mobile |  Electronics|32000.0|  05/01/2024|   Completed|       37760.0|
|  ORD003|       C003| Aman|Bangalore|  Laptop|  Electronics|55000.0|  2024/01/06|   Completed|       64900.0|
|  ORD004|       C004|Pooja|    Delhi|  Tablet| Electronics |   NULL|  2024-01-07|   Cancelled|          NULL|
|  ORD005|       C005| Neha|  Chennai|  Laptop|  Electronics|48000.0|invalid_date|   Completed|       56640.0|
+--------+-----------+-----+---------+--------+-------------+-------+------------+------------+--------------+
o

3. Add a column price_category (Low / Medium / High)

In [102]:
df=df.withColumn("price_category",
                 when(col("price")<30000,"Low")
                 .when(col("price").between(30000,50000),"Medium")
                 .otherwise("High")
                 )
df.show()

+--------+-----------+-------+---------+--------+-------------+-------+------------+------------+--------------+--------------+
|order_id|customer_id|   name|     city| product|     category|  price|  order_date|order_status|price_with_tax|price_category|
+--------+-----------+-------+---------+--------+-------------+-------+------------+------------+--------------+--------------+
|  ORD001|       C001|   Ravi|   Delhi |  Laptop|  Electronics|45000.0|  2024-01-05|   Completed|       53100.0|        Medium|
|  ORD002|       C002|  Sneha|   Mumbai| Mobile |  Electronics|32000.0|  05/01/2024|   Completed|       37760.0|        Medium|
|  ORD003|       C003|   Aman|Bangalore|  Laptop|  Electronics|55000.0|  2024/01/06|   Completed|       64900.0|          High|
|  ORD004|       C004|  Pooja|    Delhi|  Tablet| Electronics |   NULL|  2024-01-07|   Cancelled|          NULL|          High|
|  ORD005|       C005|   Neha|  Chennai|  Laptop|  Electronics|48000.0|invalid_date|   Completed|       

#Data Cleaning

4. Trim and standardize city , product , category

In [103]:
df = (df
      .withColumn("city", trim(lower(col("city"))))
      .withColumn("product", trim(lower(col("product"))))
      .withColumn("category", trim(lower(col("category"))))
)
df.show()

+--------+-----------+-------+---------+-------+-----------+-------+------------+------------+--------------+--------------+
|order_id|customer_id|   name|     city|product|   category|  price|  order_date|order_status|price_with_tax|price_category|
+--------+-----------+-------+---------+-------+-----------+-------+------------+------------+--------------+--------------+
|  ORD001|       C001|   Ravi|    delhi| laptop|electronics|45000.0|  2024-01-05|   Completed|       53100.0|        Medium|
|  ORD002|       C002|  Sneha|   mumbai| mobile|electronics|32000.0|  05/01/2024|   Completed|       37760.0|        Medium|
|  ORD003|       C003|   Aman|bangalore| laptop|electronics|55000.0|  2024/01/06|   Completed|       64900.0|          High|
|  ORD004|       C004|  Pooja|    delhi| tablet|electronics|   NULL|  2024-01-07|   Cancelled|          NULL|          High|
|  ORD005|       C005|   Neha|  chennai| laptop|electronics|48000.0|invalid_date|   Completed|       56640.0|        Medium|


5. Convert price to integer

In [104]:
df = df.withColumn("price", col("price").cast(IntegerType()))
df.show()

+--------+-----------+-------+---------+-------+-----------+-----+------------+------------+--------------+--------------+
|order_id|customer_id|   name|     city|product|   category|price|  order_date|order_status|price_with_tax|price_category|
+--------+-----------+-------+---------+-------+-----------+-----+------------+------------+--------------+--------------+
|  ORD001|       C001|   Ravi|    delhi| laptop|electronics|45000|  2024-01-05|   Completed|       53100.0|        Medium|
|  ORD002|       C002|  Sneha|   mumbai| mobile|electronics|32000|  05/01/2024|   Completed|       37760.0|        Medium|
|  ORD003|       C003|   Aman|bangalore| laptop|electronics|55000|  2024/01/06|   Completed|       64900.0|          High|
|  ORD004|       C004|  Pooja|    delhi| tablet|electronics| NULL|  2024-01-07|   Cancelled|          NULL|          High|
|  ORD005|       C005|   Neha|  chennai| laptop|electronics|48000|invalid_date|   Completed|       56640.0|        Medium|
|  ORD006|      

6. Handle invalid and null prices

In [105]:
df = df.filter(col("price").isNotNull())
df.show()

+--------+-----------+-------+---------+-------+-----------+-----+------------+------------+--------------+--------------+
|order_id|customer_id|   name|     city|product|   category|price|  order_date|order_status|price_with_tax|price_category|
+--------+-----------+-------+---------+-------+-----------+-----+------------+------------+--------------+--------------+
|  ORD001|       C001|   Ravi|    delhi| laptop|electronics|45000|  2024-01-05|   Completed|       53100.0|        Medium|
|  ORD002|       C002|  Sneha|   mumbai| mobile|electronics|32000|  05/01/2024|   Completed|       37760.0|        Medium|
|  ORD003|       C003|   Aman|bangalore| laptop|electronics|55000|  2024/01/06|   Completed|       64900.0|          High|
|  ORD005|       C005|   Neha|  chennai| laptop|electronics|48000|invalid_date|   Completed|       56640.0|        Medium|
|  ORD007|       C007|  Kiran|bangalore| tablet|electronics|30000|  2024-01-08|   Completed|       35400.0|        Medium|
|  ORD008|      

7. Normalize all dates into DateType

In [106]:
from pyspark.sql.functions import col, coalesce, try_to_timestamp, trim, regexp_replace, lit
from pyspark.sql.types import DateType

# Introduce a temporary Column expression for standardized date strings (replacing slashes/dots with hyphens)
temp_order_date_str = regexp_replace(trim(col("order_date").cast("string")), r"[./]", "-")

df = df.withColumn(
    "order_date",
    coalesce(
        try_to_timestamp(temp_order_date_str, lit("yyyy-MM-dd")),
        try_to_timestamp(temp_order_date_str, lit("dd-MM-yyyy")),
        try_to_timestamp(temp_order_date_str, lit("MM-dd-yyyy"))
    ).cast(DateType()) # Cast the final result to DateType
)

# Optional: Filter out rows where date parsing failed (order_date is NULL)
df = df.filter(col("order_date").isNotNull())

df.show(truncate=False)
df.printSchema()

+--------+-----------+-------+---------+-------+-----------+-----+----------+------------+--------------+--------------+
|order_id|customer_id|name   |city     |product|category   |price|order_date|order_status|price_with_tax|price_category|
+--------+-----------+-------+---------+-------+-----------+-----+----------+------------+--------------+--------------+
|ORD001  |C001       |Ravi   |delhi    |laptop |electronics|45000|2024-01-05|Completed   |53100.0       |Medium        |
|ORD002  |C002       |Sneha  |mumbai   |mobile |electronics|32000|2024-01-05|Completed   |37760.0       |Medium        |
|ORD003  |C003       |Aman   |bangalore|laptop |electronics|55000|2024-01-06|Completed   |64900.0       |High          |
|ORD007  |C007       |Kiran  |bangalore|tablet |electronics|30000|2024-01-08|Completed   |35400.0       |Medium        |
|ORD008  |C008       |Amit   |delhi    |laptop |electronics|45000|2024-01-09|Completed   |53100.0       |Medium        |
|ORD009  |C009       |Priya  |pu

8. Remove duplicate orders

In [107]:
df = df.dropDuplicates(["order_id"])
df.show()

+--------+-----------+-------+---------+-------+-----------+-----+----------+------------+--------------+--------------+
|order_id|customer_id|   name|     city|product|   category|price|order_date|order_status|price_with_tax|price_category|
+--------+-----------+-------+---------+-------+-----------+-----+----------+------------+--------------+--------------+
|  ORD001|       C001|   Ravi|    delhi| laptop|electronics|45000|2024-01-05|   Completed|       53100.0|        Medium|
|  ORD002|       C002|  Sneha|   mumbai| mobile|electronics|32000|2024-01-05|   Completed|       37760.0|        Medium|
|  ORD003|       C003|   Aman|bangalore| laptop|electronics|55000|2024-01-06|   Completed|       64900.0|          High|
|  ORD007|       C007|  Kiran|bangalore| tablet|electronics|30000|2024-01-08|   Completed|       35400.0|        Medium|
|  ORD008|       C008|   Amit|    delhi| laptop|electronics|45000|2024-01-09|   Completed|       53100.0|        Medium|
|  ORD009|       C009|  Priya|  

9. Filter only Completed orders

In [108]:
df = df.filter(col("order_status")=="Completed")
df.show()

+--------+-----------+-------+---------+-------+-----------+-----+----------+------------+--------------+--------------+
|order_id|customer_id|   name|     city|product|   category|price|order_date|order_status|price_with_tax|price_category|
+--------+-----------+-------+---------+-------+-----------+-----+----------+------------+--------------+--------------+
|  ORD001|       C001|   Ravi|    delhi| laptop|electronics|45000|2024-01-05|   Completed|       53100.0|        Medium|
|  ORD002|       C002|  Sneha|   mumbai| mobile|electronics|32000|2024-01-05|   Completed|       37760.0|        Medium|
|  ORD003|       C003|   Aman|bangalore| laptop|electronics|55000|2024-01-06|   Completed|       64900.0|          High|
|  ORD007|       C007|  Kiran|bangalore| tablet|electronics|30000|2024-01-08|   Completed|       35400.0|        Medium|
|  ORD008|       C008|   Amit|    delhi| laptop|electronics|45000|2024-01-09|   Completed|       53100.0|        Medium|
|  ORD009|       C009|  Priya|  

#Data Transformation

10. Create order_year , order_month

In [109]:
df = df.withColumn("order_year", year(col("order_date")))
df = df.withColumn("order_month", month(col("order_date")))
df.show()

+--------+-----------+-------+---------+-------+-----------+-----+----------+------------+--------------+--------------+----------+-----------+
|order_id|customer_id|   name|     city|product|   category|price|order_date|order_status|price_with_tax|price_category|order_year|order_month|
+--------+-----------+-------+---------+-------+-----------+-----+----------+------------+--------------+--------------+----------+-----------+
|  ORD001|       C001|   Ravi|    delhi| laptop|electronics|45000|2024-01-05|   Completed|       53100.0|        Medium|      2024|          1|
|  ORD002|       C002|  Sneha|   mumbai| mobile|electronics|32000|2024-01-05|   Completed|       37760.0|        Medium|      2024|          1|
|  ORD003|       C003|   Aman|bangalore| laptop|electronics|55000|2024-01-06|   Completed|       64900.0|          High|      2024|          1|
|  ORD007|       C007|  Kiran|bangalore| tablet|electronics|30000|2024-01-08|   Completed|       35400.0|        Medium|      2024|     

11. Aggregate total revenue per city

In [110]:
from pyspark.sql.functions import sum, aggregate
revenue_city = df.groupBy("city").agg(sum("price").alias("total_revenue"))
revenue_city.show()

+---------+-------------+
|     city|total_revenue|
+---------+-------------+
|  chennai|        83000|
|    delhi|       172000|
|bangalore|       203000|
|   mumbai|       154000|
|     pune|        28000|
+---------+-------------+



12. Aggregate total revenue per product

In [111]:
revenue_product = df.groupBy("product").agg(sum("price").alias("total_revenue"))
revenue_product.show()

+-------+-------------+
|product|total_revenue|
+-------+-------------+
| mobile|       146000|
| tablet|       124000|
| laptop|       370000|
+-------+-------------+



13. Identify top 3 cities by revenue

In [112]:
top_cities = revenue_city.orderBy(col("total_revenue").desc()).limit(3)
top_cities.show()

+---------+-------------+
|     city|total_revenue|
+---------+-------------+
|bangalore|       203000|
|    delhi|       172000|
|   mumbai|       154000|
+---------+-------------+



14. Identify products with average price above threshold

In [113]:
high_avg_price = df.groupBy("product").agg(avg("price").alias("avg_price"))\
.filter(col("avg_price")>30000)
high_avg_price.show()

+-------+------------------+
|product|         avg_price|
+-------+------------------+
| tablet|           31000.0|
| laptop|52857.142857142855|
+-------+------------------+



#File Format Operations

In [114]:
num_partitions = df.rdd.getNumPartitions()
print(f"Number of partitions: {num_partitions}")

Number of partitions: 1


In [115]:
df = df.repartition(4)

15. Write cleaned data to Parquet

In [116]:
df.write.mode("overwrite").parquet("/tmp/retail_orders_parquet")

16. Read Parquet back and verify schema

In [117]:
parquet_df = spark.read.parquet("/tmp/retail_orders_parquet")
parquet_df.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- city: string (nullable = true)
 |-- product: string (nullable = true)
 |-- category: string (nullable = true)
 |-- price: integer (nullable = true)
 |-- order_date: date (nullable = true)
 |-- order_status: string (nullable = true)
 |-- price_with_tax: double (nullable = true)
 |-- price_category: string (nullable = true)
 |-- order_year: integer (nullable = true)
 |-- order_month: integer (nullable = true)



17. Write the same data to ORC

In [118]:
df.write.mode("overwrite").orc("/tmp/retail_orders_orc")

#Performance & Validation

19. Check number of partitions

In [120]:
num_partitions = df.rdd.getNumPartitions()
print(f"Number of partitions: {num_partitions}")

Number of partitions: 4


20. Repartition before writing

In [121]:
df = df.repartition(4)

21. Compare file counts between Parquet and ORC

In [123]:
import os

parquet_path = "/tmp/retail_orders_parquet"
orc_path = "/tmp/retail_orders_orc"

def count_data_files(path):
    if not os.path.exists(path):
        return 0
    files = os.listdir(path)
    # Filter out Spark's metadata files like _SUCCESS, _common_metadata, _metadata
    data_files = [f for f in files if not f.startswith('_')]
    return len(data_files)

parquet_file_count = count_data_files(parquet_path)
orc_file_count = count_data_files(orc_path)

print(f"Number of Parquet data files: {parquet_file_count}")
print(f"Number of ORC data files: {orc_file_count}")

Number of Parquet data files: 9
Number of ORC data files: 9


22. Run explain(True) on final pipeline

In [124]:
df.explain(True)

== Parsed Logical Plan ==
Repartition 4, true
+- Repartition 4, true
   +- Project [order_id#11746, customer_id#11747, name#11748, city#11851, product#11852, category#11853, price#11888, order_date#11957, order_status#11754, price_with_tax#11784, price_category#11816, order_year#12260, month(order_date#11957) AS order_month#12261]
      +- Project [order_id#11746, customer_id#11747, name#11748, city#11851, product#11852, category#11853, price#11888, order_date#11957, order_status#11754, price_with_tax#11784, price_category#11816, year(order_date#11957) AS order_year#12260]
         +- Filter (order_status#11754 = Completed)
            +- Deduplicate [order_id#11746]
               +- Filter isnotnull(order_date#11957)
                  +- Project [order_id#11746, customer_id#11747, name#11748, city#11851, product#11852, category#11853, price#11888, cast(coalesce(try_to_timestamp(regexp_replace(trim(cast(order_date#11753 as string), None), [./], -, 1), Some(yyyy-MM-dd), TimestampType, 